#Defining class `Var`

In [ ]:
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None  #Initialize to None, which means it's not yet evaluated

    def grad(self):
        #recurse only if the value is not yet cached
        if self.grad_value is None:
            #calculate derivative using chain rule
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
    
    def __str__(self):
        return str(self.value)

    def __mul__(self, other): # z=x*y    dz/dx=y  dz/dy=x
        z = Var(self.value * other.value)
        self.children.append((other.value, z)) #x.children.append((y.value,z))  x.child =[dz/dx=y, z] <--assign children z to x
        other.children.append((self.value, z))  #y.children.append((x.value,z)) y.child =[dz/dy=x,z] <-- assign children z to y
        return z

    def __add__(self, other): #z=x+y, dz/dx=1, dz/dy=1
        z=Var(self.value+other.value)
        self.children.append((1.0,z)) #x.child =[dz/dx=1, z]
        other.children.append((1.0,z)) #y.child =[dz/dy=1, z]
        return z



def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z




In [ ]:
# Tests
Var(1) + Var(1) / Var(1) - Var(1)**Var(1)


TypeError: ignored

#Forward mode computation

In [ ]:


x=Var(0.5)
y=Var(4.2)
a=x*y
b=sin(x)
z=a+b

def printGradValue():  
    print(f'x= {x.value } dz/dx={x.grad_value}')
    print(f'y= {y.value } dz/dy={y.grad_value}')
    print(f'a= {a.value } dz/da={a.grad_value}')
    print(f'b= {b.value } dz/db={b.grad_value}')
    print(f'z= {z.value } dz/dz={z.grad_value}')


print(f'\n{x.children[0][0]},{x.children[0][1]}  <-- This is first child of x which is a. da/dx=4.2 with  a=4.2*0.5=2.1')
print(f'{x.children[1][0]:.2f},{x.children[1][1]}  <-- This is second child of x which is b. db/dx=cos(0.5)=0.87 with b=sin(0.5)=0.479\n')

print(f'{y.children[0][0]},{y.children[0][1]} <-- y only has 1 child which is a. da/dy=0.5 with  a=4.2*0.5=2.1\n')

print(f'{a.children[0][0]},{a.children[0][1]} <-- a only has 1 child which is z. dz/da=1 with  z=a*b=2.57..\n')
print(f'{b.children[0][0]},{b.children[0][1]} <-- b too only has 1 child which is z. dz/db=1 with  z=a*b=2.57..\n')



4.2,2.1  <-- This is first child of x which is a. da/dx=4.2 with  a=4.2*0.5=2.1
0.88,0.479425538604203  <-- This is second child of x which is b. db/dx=cos(0.5)=0.87 with b=sin(0.5)=0.479

0.5,2.1 <-- y only has 1 child which is a. da/dy=0.5 with  a=4.2*0.5=2.1

1.0,2.579425538604203 <-- a only has 1 child which is z. dz/da=1 with  z=a*b=2.57..

1.0,2.579425538604203 <-- b too only has 1 child which is z. dz/db=1 with  z=a*b=2.57..



#Reverse mode computation

So far we have done forward computing as we go. But we haven't computed $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$ 

In [ ]:
print('\n --Before seeding--')
printGradValue()# This should be None

print('\n --After seeding--')
z.grad_value=1.0 #Seeding #dz/dz =1
x.grad()
y.grad()

printGradValue() #Only after seeding and running .grad() method this has value




 --Before seeding--
x= 0.5 dz/dx=5.077582561890373
y= 4.2 dz/dy=None
a= 2.1 dz/da=1.0
b= 0.479425538604203 dz/db=1.0
z= 2.579425538604203 dz/dz=1.0

 --After seeding--
x= 0.5 dz/dx=5.077582561890373
y= 4.2 dz/dy=0.5
a= 2.1 dz/da=1.0
b= 0.479425538604203 dz/db=1.0
z= 2.579425538604203 dz/dz=1.0
